# OpenSearch MCP Server with Strands Agents

This notebook demonstrates how to set up and use OpenSearch MCP server in stdio and streaming (SSE/Streamable HTTP) protocols with Strands Agents, including server configuration and tool execution. For complete documentation, see [opensearch-mcp-server-py](https://github.com/opensearch-project/opensearch-mcp-server-py) repository.

## Table of Contents

- [Prerequisites](#Prerequisites)
- Basic Setup
  1. [Standard I/O (stdio) Server](#Standard-I/O-(stdio)-Server)
  2. [Streamable HTTP](#Streamable-HTTP)
  3. [Server-Sent Events (SSE)](#Server-Sent-Events-(SSE))
- Advanced Features
  - [Multi-cluster Support](#Multi-cluster-Support)
  - [Tool Filter](#Tool-Filter)
- [Clean Up](#Clean-Up)

## Prerequisites

### 1. Install required packages

In [ ]:
%pip install strands-agents --quiet
%pip install strands-agents-tools --quiet

### 2. Configure AWS credentials for Amazon Bedrock model

Strands Agents supports many different model providers. In this notebook, we'll be using the default Amazon Bedrock model provider with Claude 3.7 model.

In [ ]:
import os

os.environ['AWS_REGION'] = "<your_aws_region>"
os.environ['AWS_ACCESS_KEY_ID'] = "<your_aws_access_key>"
os.environ['AWS_SECRET_ACCESS_KEY'] = "<your_aws_secret_access_key>"
os.environ['AWS_SESSION_TOKEN'] = "<your_aws_session_token>"  # optional

## Standard I/O (stdio) Server

Configure OpenSearch MCP server with `stdio` protocol and interact with the agent using the code below. Type `/quit` to exit.

In [ ]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

# Connect to an OpenSearch MCP server using stdio transport
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx", 
        args=["opensearch-mcp-server-py"],
        env={
            "OPENSEARCH_URL": "<your_opensearch_domain_url>",
            "OPENSEARCH_USERNAME": "<your_opensearch_domain_username>",
            "OPENSEARCH_PASSWORD": "<your_opensearch_domain_password>",
        }
    )
))

# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools)

    while True:
        try:
            user_input = input("\n> ")
            if user_input == "/quit":
                break
            agent(user_input)

        except KeyboardInterrupt:
            print("\n\nExecution interrupted. Exiting...")
            break
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please try a different request.")

## Streamable HTTP

Follow these steps to setup OpenSearch MCP server with `streamable HTTP` protocol:

### 1. Install `opensearch-mcp-server-py` package

Opensearch-mcp-server-py can be installed from [PyPI](https://pypi.org/project/opensearch-mcp-server-py/) via pip:

In [ ]:
%pip install opensearch-mcp-server-py --quiet

### 2. Set preferred authentication

Both basic authentication and IAM authentication can be configured via either global environment variables or environment variables in agent MCP config file.

#### a. Basic Authentication

In [ ]:
import os

os.environ['OPENSEARCH_URL'] = "<your_opensearch_domain_url>"
os.environ['OPENSEARCH_USERNAME'] = "<your_opensearch_domain_username>"
os.environ['OPENSEARCH_PASSWORD'] = "<your_opensearch_domain_password>"

#### b. IAM Role Authentication

In [ ]:
import os

os.environ['OPENSEARCH_URL'] = "<your_opensearch_domain_url>"
os.environ['AWS_REGION'] = "<your_aws_region>"
os.environ['AWS_ACCESS_KEY_ID'] = "<your_aws_access_key>"
os.environ['AWS_SECRET_ACCESS_KEY'] = "<your_aws_secret_access_key>"
os.environ['AWS_SESSION_TOKEN'] = "<your_aws_session_token>"

#### c. OpenSearch Serverless

In [ ]:
import os

os.environ['OPENSEARCH_URL'] = "<your_opensearch_domain_url>"
os.environ['AWS_OPENSEARCH_SERVERLESS'] = True
os.environ['AWS_REGION'] = "<your_aws_region>"
os.environ['AWS_ACCESS_KEY_ID'] = "<your_aws_access_key>"
os.environ['AWS_SECRET_ACCESS_KEY'] = "<your_aws_secret_access_key>"
os.environ['AWS_SESSION_TOKEN'] = "<your_aws_session_token>"

### 3. Run the server

By default, the server will run on port **9900**. You can change the port number by specifying the port number in below command, for example:

`["python", "-m", "mcp_server_opensearch", "--transport", "stream", "--port", "<your_desired_port_number>"]`

In [ ]:
import subprocess
process = subprocess.Popen(
    ["python", "-m", "mcp_server_opensearch", "--transport", "stream"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

> Note: The server will keep running in the background, so you can continue using the notebook.

### 4. Configure Strands Agents

Configure OpenSearch MCP server with `streamable HTTP` protocol and interact with the agent using the code below. Type `/quit` to exit.

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.tools.mcp.mcp_client import MCPClient

streamable_http_mcp_client = MCPClient(lambda: streamablehttp_client("http://localhost:9900/mcp"))

# Create an agent with MCP tools
with streamable_http_mcp_client:
    # Get the tools from the MCP server
    tools = streamable_http_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools)

    while True:
        try:
            user_input = input("\n> ")
            if user_input == "/quit":
                break
            agent(user_input)

        except KeyboardInterrupt:
            print("\n\nExecution interrupted. Exiting...")
            break
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please try a different request.")

## Server-Sent Events (SSE)

Complete the steps (1-3) from Streamable HTTP section, then configure OpenSearch MCP server with `sse` protocol and interact with the agent using the code below. Type `/quit` to exit.

In [ ]:
from mcp.client.sse import sse_client
from strands import Agent
from strands.tools.mcp import MCPClient

# Connect to an MCP server using SSE transport
sse_mcp_client = MCPClient(lambda: sse_client("http://localhost:9900/sse"))

# Create an agent with MCP tools
with sse_mcp_client:
    # Get the tools from the MCP server
    tools = sse_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools)

    while True:
        try:
            user_input = input("\n> ")
            if user_input == "/quit":
                break
            agent(user_input)

        except KeyboardInterrupt:
            print("\n\nExecution interrupted. Exiting...")
            break
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please try a different request.")

## Multi-cluster Support

OpenSearch MCP server operates in either `single` (default) or `multi` model. Whiel single model connects to one cluster, multi model enables connections to multiple clusters. For more detailed information, see [Server Modes](https://github.com/opensearch-project/opensearch-mcp-server-py/blob/main/USER_GUIDE.md#server-modes).

Follow these steps to configure an OpenSearch MCP server with multi-cluster support:

### 1. Configure a YAML file with clusters information

This code creates a local `config.yml` file. You can skip this step and use your existing config file or see [example_clusters.yml](https://github.com/opensearch-project/opensearch-mcp-server-py/blob/main/example_clusters.yml) for more configuration examples.

In [ ]:
# Create config file
config = """
version: "1.0"
description: "OpenSearch cluster configurations"

clusters:
  local-cluster:
    opensearch_url: "<your_local_cluster_url>"
    opensearch_username: "<your_local_cluster_username>"
    opensearch_password: "<your_local_cluster_password>"

  remote-cluster:
    opensearch_url: "<your_remote_cluster_url>"
    opensearch_username: "<your_remote_cluster_username>"
    opensearch_password: "<your_remote_cluster_password"
"""

# Write to file
with open('config.yml', 'w') as f:
    f.write(config)

### 2. Start the server in `multi` mode

Configure OpenSearch MCP server with `stdio` protocol in `multi` mode and interact with the agent using the code below. Type `/quit` to exit.

In [ ]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

# Connect to an OpenSearch MCP server using stdio transport in multi mode
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx", 
        args=[
            "opensearch-mcp-server-py",
            "--mode", "multi",
            "--config", "config.yml"
        ]
    )
))

# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools)

    while True:
        try:
            user_input = input("\n> ")
            if user_input == "/quit":
                break
            agent(user_input)

        except KeyboardInterrupt:
            print("\n\nExecution interrupted. Exiting...")
            break
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please try a different request.")

Start the server with `streamable HTTP` or `SSE` protocol in `multi` mode:

In [ ]:
import subprocess
process = subprocess.Popen(
    ["python", "-m", "mcp_server_opensearch", "--mode", "multi", "--config", "config.yml", "--transport", "stream"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

Then, use the same agent interaction code as shown in the [Streamable HTTP](#Streamable-HTTP) or [Server-Sent Events (SSE)](#Server-Sent-Events-(SSE)) section above.

## Tool Filter

OpenSearch MCP server supports tool filtering feature where you can disable specific tools by name, category, or operation type. This can be configured using either a YAML configuration file or environment variables. In this notebook, we will use the YAML file approach. For more detailed information, see [Tool Filter](https://github.com/opensearch-project/opensearch-mcp-server-py/blob/main/USER_GUIDE.md#tool-filter).

Follow these steps to configure an OpenSearch MCP server with tool filter:

### 1. Configure a YAML file with tool filter configuration

This example creates a `non-critical` category of tools that will be disabled. Customize the categories and filters based on your requirements.

In [ ]:
# Create config file
config = """
tool_category:
  non-critical:
    - ExplainTool
    - CountTool

tool_filters:
  disabled_categories:
    - non-critical
"""

# Write to file
with open('config.yml', 'w') as f:
    f.write(config)

### 2. Start the server with tool filter configuration

Configure OpenSearch MCP server with `stdio` protocol with tool filtering enabled. The agent will only have access to non-disabled tools. Type `/quit` to exit.

In [ ]:
from mcp import stdio_client, StdioServerParameters
from strands import Agent
from strands.tools.mcp import MCPClient

# Connect to an OpenSearch MCP server using stdio transport with tool filter configuration file
stdio_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx", 
        args=[
            "opensearch-mcp-server-py",
            "--config", "config.yml"
        ],
        env={
            "OPENSEARCH_URL": "<your_opensearch_domain_url>",
            "OPENSEARCH_USERNAME": "<your_opensearch_domain_username>",
            "OPENSEARCH_PASSWORD": "<your_opensearch_domain_password>"
        }
    )
))

# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(tools=tools)

    while True:
        try:
            user_input = input("\n> ")
            if user_input == "/quit":
                break
            agent(user_input)

        except KeyboardInterrupt:
            print("\n\nExecution interrupted. Exiting...")
            break
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please try a different request.")

Start the server with `streamable HTTP` or `SSE` protocol, specifying the tool configuration file:

In [ ]:
import subprocess
process = subprocess.Popen(
    ["python", "-m", "mcp_server_opensearch", "--config", "config.yml", "--transport", "stream"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

Then, use the same agent interaction code as shown in the [Streamable HTTP](#Streamable-HTTP) or [Server-Sent Events (SSE)](#Server-Sent-Events-(SSE)) section above.

## Clean Up

### Stop server

In [ ]:
# Graceful termination
process.terminate()
process.wait()

### Remove configuration file

In [ ]:
# Clean up config file
import os

try:
    os.remove('config.yml')
    print("Config file removed successfully")
except FileNotFoundError:
    print("Config file already removed")